In [36]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import HuggingFaceHub
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
import os 
from dotenv import load_dotenv
from time import time
import warnings
warnings.filterwarnings('ignore')

In [37]:
#loader = DirectoryLoader('PDF_Testing', glob="./*.pdf", loader_cls=PyPDFLoader)
loader = PyPDFLoader('How AI is expanding art history.pdf')
documents = loader.load()

In [38]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [39]:
len(texts)

139

In [40]:
texts[5]

Document(page_content='advancing fast, and dozens of studies are now \nproving the power of AI to shed new light on', metadata={'source': 'How AI is expanding art history.pdf', 'page': 0})

In [41]:
_ = load_dotenv()

HUGGINGFACEHUB_API_TOKEN = os.environ["HUGGINGFACEHUB_API_TOKEN"]

llm=HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta", 
    model_kwargs={"temperature":0.2, "max_length":256},
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN
    )

In [42]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="BAAI/bge-base-en-v1.5",
                                                      model_kwargs={"device": "cuda"})

load INSTRUCTOR_Transformer
max_seq_length  512


In [43]:
%%time
persist_directory = 'db_HuggingFace'

embedding = instructor_embeddings

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

CPU times: total: 172 ms
Wall time: 686 ms


In [44]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})
docs = retriever.get_relevant_documents("What is paranoia?")

In [45]:
len(docs)

2

In [46]:
qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [47]:
def process_llm_response(qa_chain, query):
    print(f"Query: {query}\n")
    time_1 = time()
    llm_response = qa_chain(query)
    time_2 = time()
    print(f"Inference time: {round(time_2-time_1, 3)} sec.")
    print(f"\nResult:", llm_response['result'])
    print(f"\nmetadata:", llm_response['source_documents'][0].metadata)

In [49]:
query = "How does AI affect Art?"
process_llm_response(qa_chain, query)

Query: How does AI affect Art?

Inference time: 7.643 sec.

Result:  In the study of art, AI is revealing new insights into color and style by analyzing large datasets of artworks. AI-driven tools are helping artists understand the science behind their creative choices, allowing them to make more informed decisions in their work. However, to fully harness the power of AI in art, we will need the same foundations as other fields, such as data privacy and transparency in algorithmic decision-making. Overall, AI is enhancing the creative process in art by providing new perspectives and tools for exploration.

metadata: {'page': 2, 'source': 'How AI is expanding art history.pdf'}


In [50]:
# break it down
query = "How does AI mimic human artwork?"
process_llm_response(qa_chain, query)

Query: How does AI mimic human artwork?

Inference time: 8.009 sec.

Result:  AI-driven tools are revealing how artists' understanding of the science of color and style contributes to their unique artistic style, allowing for the creation of AI-generated artwork that mimics the style of human artists. This is made possible by the same foundations required for the full power of AI in the study of art.

metadata: {'page': 2, 'source': 'How AI is expanding art history.pdf'}


In [15]:
query = "How do they measure Momentary social anxiety?"
process_llm_response(qa_chain, query)

Query: How do they measure Momentary social anxiety?

Inference time: 5.45 sec.

Result:  The study assessed momentary social anxiety using three specific items suggested in a previous study. Unfortunately, the specific items are not provided in this context, so the exact measurement method is not clear. However, it can be inferred that these items likely focus on measuring the intensity and duration of social anxiety in specific situations or interactions. Without further information, it is impossible to provide a more detailed answer.

metadata: {'page': 5, 'source': 'PDF_file\\s41598-023-47912-0.pdf'}


In [51]:
query = "Please give me an example of using AI to produce an artwork"
process_llm_response(qa_chain, query)

Query: Please give me an example of using AI to produce an artwork

Inference time: 7.114 sec.

Result:  One example of using AI to produce an artwork is the "Portrait of Edmond de Belamy," created by a team of researchers from the French company, Obvious. The portrait was generated using a generative adversarial network (GAN), a type of AI that can create new images by learning from existing ones. The GAN was trained on a dataset of over 15,000 portraits from the 14th to the 20th centuries, allowing it to generate a unique portrait that appears to have a human-like quality. The portrait was sold at an auction for over $400,000, highlighting the growing interest and value in AI-generated art.

metadata: {'page': 0, 'source': 'How AI is expanding art history.pdf'}


In [52]:
query = "How does machine learning and deep neural network help developing AI in art field?"
process_llm_response(qa_chain, query)

Query: How does machine learning and deep neural network help developing AI in art field?

Inference time: 5.916 sec.

Result:  Machine learning and deep neural networks are crucial foundations for realizing the full potential of AI in the study of art. By analyzing vast amounts of visual data, these technologies can help us understand the underlying principles of color, composition, and style in art. AI-driven tools are already revealing new insights into how artists' understanding of the science of color and style evolves over time, and how these principles are applied in different artistic traditions. As these technologies continue to advance, they will enable us to create more sophisticated and nuanced AI systems for generating, analyzing, and interpreting art.

metadata: {'page': 2, 'source': 'How AI is expanding art history.pdf'}


In [53]:
query = "What is By-eye art analysis?"
process_llm_response(qa_chain, query)

Query: What is By-eye art analysis?

Inference time: 5.876 sec.

Result:  By-eye art analysis is a technique that involves closely examining and interpreting visual and stylistic elements in artworks, without the use of technology or scientific tools. It is a subjective and interpretive process that allows art scholars to gain insights into the meaning and significance of the artwork. By-eye art analysis can vary depending on the individual's background, training, and perspective, but it is becoming increasingly important in the field of art scholarship as new technologies and scientific methods are being developed to supplement or replace traditional methods of analysis.

metadata: {'page': 1, 'source': 'How AI is expanding art history.pdf'}


In [54]:
query = "Other than produce Artwork, what else can AI use in Art study?"
process_llm_response(qa_chain, query)

Query: Other than produce Artwork, what else can AI use in Art study?

Inference time: 3.918 sec.

Result:  AI can be used to analyze and interpret art pieces, providing insights into the techniques used by the artist, the historical context of the artwork, and even the emotions conveyed in the piece. This can help art historians and curators better understand and appreciate the artworks in their collections. Additionally, AI can be used to assist in the restoration and conservation of art pieces, by identifying areas of damage and suggesting appropriate restoration methods.

metadata: {'page': 2, 'source': 'How AI is expanding art history.pdf'}


In [56]:
query = "Give me an example of AI help recovering old art pieces"
process_llm_response(qa_chain, query)

Query: Give me an example of AI help recovering old art pieces

Inference time: 7.858 sec.

Result:  One example of AI helping to recover old art pieces is the use of deep learning algorithms to analyze fragments of paintings and sculptures that have been damaged or destroyed over time. These algorithms can reconstruct the missing parts of the artwork based on the surrounding context, helping to restore the original piece to its former glory. This technology has been used to restore works by famous artists such as Leonardo da Vinci and Michelangelo, as well as lesser-known pieces from various historical periods.

metadata: {'page': 0, 'source': 'How AI is expanding art history.pdf'}


In [58]:
query = "How many poses and gender across more than 20,000 portraits?"
process_llm_response(qa_chain, query)

Query: How many posed and gender has the AI been analyzed?

Inference time: 2.282 sec.

Result:  According to the given context, there are no mentions of how many posed and genderless AI have been analyzed. Therefore, the answer is that we don't know.

metadata: {'page': 3, 'source': 'PDF_file\\s41598-023-47912-0.pdf'}


In [22]:
query = "Where did the study take place?"
process_llm_response(qa_chain, query)

Query: Where did the study take place?

Inference time: 4.38 sec.

Result:  The study was supported by the Seed Funding Support for Thesis Research 2019–20, Faculty of Social Sciences. However, the question does not provide information about where the study took place. Without further context, it is unclear where the study was conducted.

metadata: {'page': 9, 'source': 'PDF_file\\s41598-023-47912-0.pdf'}


In [23]:
qa_chain.retriever.search_type , qa_chain.retriever.vectorstore

('similarity', <langchain.vectorstores.chroma.Chroma at 0x2305aa5af10>)

In [24]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.template)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:
